In [ ]:
import csv
from nltk.translate.bleu_score import sentence_bleu
import string 
import re 

import string

%pip install nltk
%pip install pandas
%pip install scikit-learn

import pandas as pd
import nltk
from nltk.corpus import wordnet

nltk.download('punkt')


from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import numpy as np
import re
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')



In [ ]:


lemmatizer = WordNetLemmatizer()

sw = stopwords.words('english')
porter_stemmer = PorterStemmer()


def text_cleaning(text):
    text = str (text)

    printable = set(string.printable)
    text = ''.join(filter(lambda x: x in printable, text))
    text = text.replace('\x00', ' ')  # remove nulls
    text = text.replace('\r', ' ')
    text = text.replace('\n', ' ')
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    text = text.lower()  # Lowercasing

    words = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    wn_tags = {'N': wordnet.NOUN, 'V': wordnet.VERB, 'R': wordnet.ADV, 'J': wordnet.ADJ}
    # lemmatize each word with its POS tag
    lemmatized_words = []
    for word, pos in pos_tags:
        if pos[0] in wn_tags:
            wn_tag = wn_tags[pos[0]]
            lemmatized_words.append(lemmatizer.lemmatize(word, wn_tag))
        else:
            lemmatized_words.append(word)
    text = ' '.join([str(elem) for elem in lemmatized_words])

    words= text.split()
    stemmed_words=[porter_stemmer.stem(word=word) for word in words] # Stemming
    text = ' '.join([str(elem) for elem in stemmed_words])

    text = text.strip()
    return text


In [ ]:
column_comments = []
column_gold = []
column_chatgpt = []
column_gpt4 = []
column_flan = []


with open('causes.csv', 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    idx = 0
    for row in csv_reader:
        column_comments.append(text_cleaning(row[1]))
        column_gold.append(text_cleaning(row[2]))
        column_gpt4.append(text_cleaning(row[3]))
        column_chatgpt.append(text_cleaning(row[4]))
        column_flan.append(text_cleaning(row[5]))

print(len(column_gold))

In [ ]:
word_counter = 0
for sentence in column_flan:
    word_counter = word_counter + len(sentence.split())
print(word_counter/len(column_flan))

In [ ]:
# ChatGPT: Calculate the BLEU score for each row in the columns
scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_chatgpt[i], weights=(0.5, 0.5))
    # print(score)
    scores.append(score)


print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)

print(f"Average BLEU score: {avg_score}")

In [ ]:
# GPT-4: Calculate the BLEU score for each row in the columns
scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_gpt4[i], weights=(0.5, 0.5))
    # print(score)
    scores.append(score)


print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)

print(f"Average BLEU score: {avg_score}")

In [ ]:
# flan-alpaca: Calculate the BLEU score for each row in the columns

scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_flan[i], weights=(0.5, 0.5))
    # print(score)
    scores.append(score)


print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)

print(f"Average BLEU score: {avg_score}")